Problem Set 1

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

We delete clients who leave the company (Churn=No)

In [29]:
df = pd.read_csv (r'customers.csv') #import data
negative_churn = df.Churn=='No' #find who leave the company (churn) 
df = df.drop(df[negative_churn].index) #delete them from the DataFrame

In [30]:
number_tot = len(negative_churn) #total number of clients
number_positive_churn = len(df) #number of clients who do NOT leave the company 
perc_positive_tot=number_positive_churn/number_tot
print('Percentage of clients who do NOT leave the company (on the total) = %.3f' % perc_positive_tot)
print('Percentage of clients who leave the company (on the total) = %.3f' % (1-perc_positive_tot))

Percentage of clients who do NOT leave the company (on the total) = 0.265
Percentage of clients who leave the company (on the total) = 0.735


In [33]:
female = df.gender=='Female' #find woman
n_female = np.sum(female) #count the True values
n_male = len(female)-n_female
perc_female = n_female/number_tot
perc_male = n_male/number_tot
#sns.barplot(x='Gender', y='percentage', data=df.gender)
